In [ ]:
import requests
import json
import csv
from datetime import datetime, timezone

#### 輸出的CSV
output_file = "reviews_output.csv"

#### ID不重複
unique_ids = set()

#### 計算評論數 & 有內文的評論數
total_reviews = 0
reviews_with_comments = 0

#### 寫入CSV標題
with open(output_file, mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Rating", "TIME", "Comment"])

#### 從CSV載入URL列表
input_file = "google_maps_review_urls.csv"
with open(input_file, mode='r', encoding='utf-8') as input_file:
    csv_reader = csv.reader(input_file)
    next(csv_reader)  
    for row in csv_reader:
        if row:  
            url = row[0]  

#### 發出請求
            response = requests.get(url)
            response.raise_for_status()

            raw_content = response.text.strip()

            #### 移除開頭的)]}'
            if raw_content.startswith(")]}'"):
                raw_content = raw_content[4:]

            data = json.loads(raw_content)

            #### 提取要的資料
            with open(output_file, mode='a', encoding='utf-8', newline='') as file:
                writer = csv.writer(file)
                for i in range(10):
                    try:                        
                        review_id =  str(data[2][i][0][0])
                        rating = str(data[2][i][0][2][0][0])
                        timestamp_seconds = data[2][i][0][1][2] // 1000000
                        formatted_date = datetime.fromtimestamp(timestamp_seconds, tz=timezone.utc).strftime('%Y-%m-%d')
                        try:
                            comment = data[2][i][0][2][15][0][0]
                        except (IndexError, TypeError):
                            comment = ""
                        # 確保 ID 唯一，不重複儲存
                        if review_id not in unique_ids:
                            writer.writerow([review_id, rating, formatted_date, comment])
                            unique_ids.add(review_id)
                            total_reviews += 1  # 增加計數
                            if comment.strip():  # 若評論內容不為空
                                    reviews_with_comments += 1
                    except Exception as e:
                        print(f"評論 {i + 1}: 無評論或格式錯誤 ({e})")

print(f"總共存入 {total_reviews} 筆評論")
print(f"其中有 {reviews_with_comments} 筆評論有內文")